In [38]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [39]:
import pandas as pd
import torch
from sklearn.preprocessing import LabelEncoder, StandardScaler
import joblib

def load_data(train_path, test_path, encoder_path=None, scaler_path=None):
    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)

    X_train = train_df.iloc[:, :-1].values
    y_train = train_df.iloc[:, -1].values
    X_test = test_df.iloc[:, :-1].values
    y_test = test_df.iloc[:, -1].values

    label_encoder = LabelEncoder()
    y_train = label_encoder.fit_transform(y_train)
    y_test = label_encoder.transform(y_test)

    # Save the encoder if a path is provided
    if encoder_path:
        joblib.dump(label_encoder, encoder_path)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Save the scaler if a path is provided
    if scaler_path:
        joblib.dump(scaler, scaler_path)

    return (
        torch.tensor(X_train, dtype=torch.float32),
        torch.tensor(y_train, dtype=torch.long),
        torch.tensor(X_test, dtype=torch.float32),
        torch.tensor(y_test, dtype=torch.long),
        len(label_encoder.classes_)
    )


In [40]:
encoder_path = "label_encoder.pkl"
scaler_path = "scaler.pkl"

In [41]:
X_train, y_train, X_test, y_test, num_classes = load_data('train_2dist.csv', 'test_2dist.csv',encoder_path=encoder_path, scaler_path=scaler_path)


In [42]:
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)


In [43]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [44]:
class DeepNet(nn.Module):
    def __init__(self, input_size, num_classes):
        super(DeepNet, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, 128),       # Input -> 128
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            nn.Linear(128, 64),               # 128 -> 64
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            nn.Linear(64, num_classes)        # 64 -> Output (num_classes)
        )

    def forward(self, x):
        return self.network(x)


In [45]:
input_size = X_train.shape[1]
model = DeepNet(input_size, num_classes)
model = model.to(device)



In [46]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)
model.apply(init_weights)


DeepNet(
  (network): Sequential(
    (0): Linear(in_features=128, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.3, inplace=False)
    (8): Linear(in_features=64, out_features=33, bias=True)
  )
)

In [47]:
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [48]:
def train_model(model, train_loader, criterion, optimizer, num_epochs=20):
    model.train()
    for epoch in range(num_epochs):
        epoch_loss = 0
        correct = 0
        total = 0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            _, predicted = outputs.max(1)
            total += y_batch.size(0)
            correct += predicted.eq(y_batch).sum().item()

        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss / len(train_loader):.4f}, Accuracy: {100. * correct / total:.2f}%")



In [49]:
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            _, predicted = outputs.max(1)
            total += y_batch.size(0)
            correct += predicted.eq(y_batch).sum().item()

    print(f"Test Accuracy: {100. * correct / total:.2f}%")



In [50]:
train_model(model, train_loader, criterion, optimizer, num_epochs=1000)
evaluate_model(model, test_loader)


Epoch 1/1000, Loss: 1.8346, Accuracy: 47.54%
Epoch 2/1000, Loss: 1.0806, Accuracy: 67.01%
Epoch 3/1000, Loss: 0.9030, Accuracy: 72.09%
Epoch 4/1000, Loss: 0.8099, Accuracy: 75.36%
Epoch 5/1000, Loss: 0.7688, Accuracy: 76.83%
Epoch 6/1000, Loss: 0.6992, Accuracy: 78.40%
Epoch 7/1000, Loss: 0.6605, Accuracy: 80.02%
Epoch 8/1000, Loss: 0.6532, Accuracy: 79.81%
Epoch 9/1000, Loss: 0.6177, Accuracy: 81.22%
Epoch 10/1000, Loss: 0.5640, Accuracy: 82.09%
Epoch 11/1000, Loss: 0.5614, Accuracy: 82.48%
Epoch 12/1000, Loss: 0.5254, Accuracy: 83.86%
Epoch 13/1000, Loss: 0.5348, Accuracy: 83.23%
Epoch 14/1000, Loss: 0.5283, Accuracy: 83.68%
Epoch 15/1000, Loss: 0.4798, Accuracy: 84.34%
Epoch 16/1000, Loss: 0.4950, Accuracy: 84.83%
Epoch 17/1000, Loss: 0.4987, Accuracy: 84.89%
Epoch 18/1000, Loss: 0.5167, Accuracy: 83.86%
Epoch 19/1000, Loss: 0.4666, Accuracy: 84.80%
Epoch 20/1000, Loss: 0.4714, Accuracy: 85.73%
Epoch 21/1000, Loss: 0.4307, Accuracy: 86.51%
Epoch 22/1000, Loss: 0.4459, Accuracy: 85.5

In [51]:
torch.save(model.state_dict(), 'model.pth')
